In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp drive/MyDrive/집중교육/집중교육\ 팀프로젝트/train_label_baseline.csv .
!cp drive/MyDrive/집중교육/집중교육\ 팀프로젝트/test_label.csv .

In [ ]:
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset0.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset1.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset2.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset3.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset45.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset6.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset7.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset89.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset10.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset11.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset12.zip -d .
!unzip drive/MyDrive/집중교육/집중교육\ 팀프로젝트/dataset13.zip -d .

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./dataset13/524bfc8159111e4571697d26793100d2_13.png  
  inflating: ./dataset13/c20ce998ba99451991c352d02061b3f7_13.png  
  inflating: ./dataset13/b9089b8cf70c95e1f4b6538438a0b930_13.png  
  inflating: ./dataset13/768648ae74e9d84e6d445fdc5fd8bfe7_13.png  
  inflating: ./dataset13/1d4be24ba28946152470de2774d6099_13.png  
  inflating: ./dataset13/1738b29e7344f8447c44435d252fe26d_13.png  
  inflating: ./dataset13/16868a30ebe634053dc22b596edc161e_13.png  
  inflating: ./dataset13/9891e489f1f4dab52056b4bd5d870b47_13.png  
  inflating: ./dataset13/d89cb5da6288ae91a21dea5979316c3e_13.png  
  inflating: ./dataset13/e76aad9a92f83d70bda72093f9b5aa73_13.png  
  inflating: ./dataset13/273e1da4dfcf82443c5e2d7fd3020266_13.png  
  inflating: ./dataset13/c0eb62cb64ee1a31f29de30ec1bf6a3_13.png  
  inflating: ./dataset13/d13986cc2403a2034b4b3d2a28039009_13.png  
  inflating: ./dataset13/31768574bcddbdde60ad823977adaa23_13.png  
  inflating: ./dataset13/f10

## Training

In [ ]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 78.9 MB/s eta 0:00:00


In [ ]:
import timm
import copy
from torchvision import transforms
from tqdm import tqdm
from PIL import Image
from fastprogress import master_bar, progress_bar
from torch.autograd import Variable
from torchvision import models
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import time
import sys
from pathlib import Path
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2
import warnings
warnings.filterwarnings("ignore")

In [ ]:
class ResNet18(nn.Module):
  def __init__(self, num_classes=55, is_trained=False):
    super().__init__()
    self.net = timm.create_model('resnet18',pretrained = True, num_classes = num_classes)
  def forward(self, inputs):
    return self.net(inputs)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
  torch.cuda.empty_cache()
print(device)

cuda


In [ ]:
class CustomDataset(Dataset):
  def __init__(self, csv_file, transform=None):
    self.data = pd.read_csv(csv_file)
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img_path = self.data.iloc[idx,1]
    # print(img_path)
    try:
     image = Image.open(img_path)
     label = self.data.iloc[idx,2]
     # print(label)
     if image.mode == 'RGBA':
         image = image.convert('RGB')
     if self.transform:
       image = self.transform(image)
     label = torch.tensor(label)
     return image, label
    except OSError:
     print(f"Skipping corrupted image: {img_path}")
     return self.__getitem__((idx + 1) % len(self.data))  # load the next image

In [ ]:
def mixup(x, y, num_classes=55):
    # assert False, "you should implement mixup first"

    # generate mixed sample
    lam = np.random.beta(1,1)
    batch_size = x.size()[0]

    index = np.random.permutation(batch_size)
    mixed_x = lam * x + ( 1 - lam ) * x[index]
    # mixed_y = lam * y + ( 1 - lam ) * y[index]

    y_onehot = torch.zeros(batch_size, num_classes)
    y_onehot.scatter_(1, y.unsqueeze(1), 1)

    mixed_y_onehot = torch.zeros(batch_size, num_classes)
    mixed_y_onehot.scatter_(1, y[index].unsqueeze(1), 1)

    mixed_y = lam * y_onehot + ( 1 - lam ) * mixed_y_onehot

    return mixed_x, mixed_y

In [ ]:
def train_model(model, train_loader, criterion, optimizer, scheduler, device, batch_size=32):
  model.train()

  total_loss =0
  epoch_start_time = time.time()
  total_correct = 0
  total_items = 0

  print("epoch: {}".format(epoch+1))

  train_progress_bar = tqdm(train_loader, desc = "Training", total=len(train_loader))

  for inputs, labels in train_progress_bar:

    total_items += inputs.size(0)

    inputs, labels = mixup(inputs, labels)

    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    scheduler.step()

    total_loss += loss.item()
    total_correct += torch.sum(preds == labels.data).cpu().numpy()

    train_progress_bar.set_postfix({"loss": loss.item(), "acc": total_correct / total_items*100})

  epoch_loss = total_loss / len(train_loader)
  epoch_acc = total_correct / total_items*100
  epoch_time = time.time() - epoch_start_time

  print("Epoch: {} Total Items: {} Training Loss : {:.4f} Accuracy : {:.4f}% Time : {:.4f}".format(epoch+1, total_items, epoch_loss, epoch_acc, epoch_time))

  return epoch_loss, epoch_acc

In [ ]:
@torch.no_grad()
def test_model(model, test_loader, criterion, device):
  model.eval()

  total_loss = 0
  total_correct = 0
  total_items = 0
  test_progress_bar = tqdm(test_loader, desc = "Testing", total=len(test_loader))
  for inputs, labels in test_progress_bar:
    total_items += inputs.size(0)
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, labels)

    total_loss += loss.item()
    total_correct += torch.sum(preds == labels.data).cpu().numpy()
    test_progress_bar.set_postfix({"loss": loss.item(), "acc": total_correct / total_items*100})

  epoch_loss = total_loss / len(test_loader)
  epoch_acc = total_correct / total_items *100
  print("Testing Total Items : {} Loss : {:.4f} Accuracy : {:.4f}%".format(total_items, epoch_loss, epoch_acc))

  return epoch_loss, epoch_acc

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [ ]:
train_data = CustomDataset("train_label_baseline.csv", train_transform)
test_data = CustomDataset("test_label.csv", test_transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=512, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=512, shuffle=False, num_workers=4)

In [ ]:
model = ResNet18()
model.to(device)

best_model = copy.deepcopy(model.state_dict())

epochs = 300

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, total_steps=epochs*len(train_loader))

best_loss = 100000
best_acc = 0

train_loss_list = []
train_acc_list = []
loss_list=[]
acc_list=[]

threshold = 0

for epoch in range(epochs):
  train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, scheduler, device)
  loss, acc = test_model(model, test_loader, criterion, device)
  train_loss_list.append(train_loss)
  train_acc_list.append(train_acc)
  loss_list.append(loss)
  acc_list.append(acc)

  if loss < best_loss:
    best_loss = loss
    best_acc = acc
    best_model = copy.deepcopy(model.state_dict())
    torch.save(best_model,'drive/MyDrive/집중교육/집중교육 팀프로젝트/'+'best_model_baseline.pth')

  if loss >= best_loss:
    threshold += 1
  else:
    threshold = 0
  if threshold == 20:
    break

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

epoch: 1


Training: 100%|██████████| 738/738 [11:47<00:00,  1.04it/s, loss=1.38, acc=49.2]


Epoch: 1 Total Items: 377658 Training Loss : 2.1541 Accuracy : 49.1969% Time : 707.2193


Testing: 100%|██████████| 287/287 [04:36<00:00,  1.04it/s, loss=4.41, acc=62.4]


...

Testing Total Items : 146902 Loss : 0.8895 Accuracy : 77.1855%
epoch: 20


Training: 100%|██████████| 738/738 [12:01<00:00,  1.02it/s, loss=0.257, acc=90.4]


Epoch: 20 Total Items: 377658 Training Loss : 0.2894 Accuracy : 90.4342% Time : 721.5821


Testing: 100%|██████████| 287/287 [04:31<00:00,  1.06it/s, loss=1.89, acc=76.9]

Testing Total Items : 146902 Loss : 0.9087 Accuracy : 76.9370%


In [ ]:
print(loss_list)
print(acc_list)
print(train_loss_list)
print(train_acc_list)

[1.4796739533699348, 1.1328165533235264, 1.017326694254468, 0.9482656183818076, 0.9086815369046108, 0.8810126732658426, 0.866439906198804, 0.8395826662874388, 0.8383298330917591, 0.8359430626761623, 0.8282609253031451, 0.8285024292273804, 0.8358527858082841, 0.862109321023529, 0.8451317435305709, 0.8611866803223248, 0.8519672603193891, 0.8968192237698658, 0.8894855663543795, 0.9086671066616471]
[62.3810431444092, 69.6239669984071, 72.09500211025038, 73.65114157737813, 74.76004411104002, 75.53334876312098, 75.65315652611946, 76.65110073382255, 76.92883691168262, 76.7729506745994, 77.23448285251392, 77.2637540673374, 77.32161577105826, 76.3468162448435, 76.7409565560714, 76.68173340049829, 77.32774230439341, 76.78996882275258, 77.18547058583273, 76.93700562279615]
[2.154054544482451, 1.1340326456035055, 0.9003731417785169, 0.7890749056003281, 0.7159584760665894, 0.6612332720097488, 0.6183679476582261, 0.5815323241115585, 0.5493928710538843, 0.5198138592850549, 0.49246510221221584, 0.4663

## Accuracy Per Categories

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [ ]:
test_data = CustomDataset("test_label.csv", test_transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=512, shuffle=False, num_workers=4)

In [ ]:
!cp /content/drive/MyDrive/집중교육/집중교육\ 팀프로젝트/best_model_baseline.pth .

In [ ]:
model = ResNet18()
model.load_state_dict(torch.load('best_model_baseline.pth'))
model.to(device)

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

ResNet18(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), 

In [ ]:
total_labels = {} # 55개 label별 개수
correct_labels = {} # 55개 label별 맞춘 개수

In [ ]:
@torch.no_grad()
def test_model(model, test_loader, criterion, device):
  model.eval()

  total_loss = 0
  total_correct = 0
  total_items = 0
  test_progress_bar = tqdm(test_loader, desc = "Testing", total=len(test_loader))
  for inputs, labels in test_progress_bar:
    total_items += inputs.size(0)
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, labels)

    total_loss += loss.item()
    total_correct += torch.sum(preds == labels.data).cpu().numpy()
    test_progress_bar.set_postfix({"loss": loss.item(), "acc": total_correct / total_items*100})

    for i in range(len(labels)):
      if labels[i].item() not in total_labels:
        total_labels[labels[i].item()] = 1
        correct_labels[labels[i].item()] = 0
      else:
        total_labels[labels[i].item()] += 1
      if labels[i].item() == preds[i].item():
        correct_labels[labels[i].item()] += 1

  epoch_loss = total_loss / len(test_loader)
  epoch_acc = total_correct / total_items *100
  print("Testing Total Items : {} Loss : {:.4f} Accuracy : {:.4f}%".format(total_items, epoch_loss, epoch_acc))

  return epoch_loss, epoch_acc

In [ ]:
criterion = nn.CrossEntropyLoss()
loss, acc = test_model(model, test_loader, criterion, device)

Testing: 100%|██████████| 287/287 [04:31<00:00,  1.06it/s, loss=1.89, acc=77.2]

Testing Total Items : 146902 Loss : 0.8282 Accuracy : 77.2311%


In [ ]:
# print accuracy per label
for label in total_labels:
  print("label: {}, total label :{}, correct :{} , accuracy : {}%".format(label, total_labels[label], correct_labels[label], correct_labels[label]/total_labels[label]*100))

label: 0, total label :11326, correct :10623 , accuracy : 93.79304255694862%
label: 1, total label :966, correct :563 , accuracy : 58.2815734989648%
label: 2, total label :238, correct :74 , accuracy : 31.092436974789916%
label: 3, total label :322, correct :66 , accuracy : 20.496894409937887%
label: 4, total label :2394, correct :1535 , accuracy : 64.1186299081036%
label: 5, total label :658, correct :281 , accuracy : 42.70516717325228%
label: 6, total label :5082, correct :3450 , accuracy : 67.88665879574971%
label: 7, total label :210, correct :48 , accuracy : 22.857142857142858%
label: 8, total label :1260, correct :501 , accuracy : 39.76190476190476%
label: 9, total label :1400, correct :1022 , accuracy : 73.0%
label: 10, total label :518, correct :339 , accuracy : 65.44401544401545%
label: 11, total label :2632, correct :2133 , accuracy : 81.04103343465046%
label: 12, total label :4396, correct :2837 , accuracy : 64.53594176524113%
label: 13, total label :322, correct :120 , accu